In [0]:
#Return Spark session
from pyspark.sql.session import SparkSession
def get_spark_session(app_name="Some Anonymous Data Engineering Project"):
    try:
        spark = SparkSession.getActiveSession()
        if spark:
            return spark
    except:
        pass

    return (SparkSession.builder.appName(app_name).getOrCreate())



In [0]:
def read_file(spark,filetype,path,header=True,infer_schema=True,mline=True):
    if filetype=="csv":
        return spark.read.csv(path,header=header,inferSchema=infer_schema)#read_csv_df(spark,path)
    elif filetype=="json":
        return read_json_df(spark,path)
    elif filetype=="delta":
        return read_delta_df(spark,path)
    elif filetype=='orc':
        return spark.read.orc(path)
    elif filetype=='parquet':
        return spark.read.parquet(path)
    else:
        raise Exception("File type not supported")

In [0]:
def write_file(df, path,format="delta", mode="overwrite"):
    return df.write.mode(mode).format(format).save(path)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window


def enrich_df(df):
    return(df.withColumn("ingestion_time",current_timestamp()))

# -----------------------------------------------------------------------------------------------------

def enrich_add_col(df, columns):
    for cname, props in columns.items():
        dtype = props.get("type")
        default = props.get("default")

        if dtype == "string":
            df = df.withColumn(cname, lit(default if default is not None else ""))
        elif dtype == "int":
            df = df.withColumn(cname, lit(default if default is not None else 0))
        elif dtype == "float":
            df = df.withColumn(cname, lit(default if default is not None else 0.0))
        elif dtype == "boolean":
            df = df.withColumn(cname, lit(default if default is not None else False))
        elif dtype == "timestamp":
            df = df.withColumn(cname,lit(default) if default is not None else current_timestamp())
        else:
            print(f"Unsupported data type: {dtype}")
    return df

# -----------------------------------------------------------------------------------------------------
# def f_cast_col(df,schema_map):
#     for cname,dtype in schema_map.items():
#         df = df.withColumn(cname, col(cname).cast(dtype))
#     return df
def f_cast_col(df, schema_map):
    for cname, dtype in schema_map.items():
        if cname in df.columns:
            df = df.withColumn(cname, col(cname).cast(dtype))
        else:
            print(f"⚠️ Column not found: {cname}")
    return df


# -----------------------------------------------------------------------------------------------------
def deduplicate_with_window(df, partition_cols, order_col=None, order_type="asc"):

    """
    Removes duplicates using window function.

    Parameters:
    df              : Input DataFrame
    partition_cols  : List of columns to identify duplicates
    order_col       : Column to decide which row to keep
    order_type      : 'asc' or 'desc' (default: asc)

    Returns:
    Deduplicated DataFrame
    """

    # Build window specification
    if order_col:
        if order_type == "desc":
            window_spec = Window.partitionBy(*partition_cols).orderBy(col(order_col).desc())
        else:
            window_spec = Window.partitionBy(*partition_cols).orderBy(col(order_col))
    else:
        window_spec = Window.partitionBy(*partition_cols).orderBy(lit(1))

    # Apply row_number
    df_rn = df.withColumn("rn", row_number().over(window_spec))

    # Keep only first record
    df_dedup = df_rn.filter(col("rn") == 1).drop("rn")

    return df_dedup

# -----------------------------------------------------------------------------------------------------
def deduplicate(df, keys):#pass the keys as a list []
    return df.dropDuplicates(keys)

# -----------------------------------------------------------------------------------------------------
def drop_required_nulls(df, columns):#pass the columns as a list []
    return df.dropna(subset=columns)
# -----------------------------------------------------------------------------------------------------
def sum_two_columns(df, col1, col2, output_col):
    return df.withColumn(output_col, col(col1) + col(col2))

# -----------------------------------------------------------------------------------------------------
def standardize_string_columns(df, rules):
    """
    rules = {
        "column_name": "operation",
        "column_name2": "operation"
    }
    """
    for column_name, operation in rules.items():

        if operation == "lower_trim":
            df = df.withColumn(column_name, lower(trim(col(column_name))))

        elif operation == "upper_trim":
            df = df.withColumn(column_name, upper(trim(col(column_name))))

        elif operation == "initcap_trim":
            df = df.withColumn(column_name, initcap(trim(col(column_name))))

        elif operation == "trim":
            df = df.withColumn(column_name, trim(col(column_name)))

        elif operation == "remove_spaces":
            df = df.withColumn(
                column_name,
                regexp_replace(col(column_name), " ", "")
            )

        elif operation == "null_if_empty":
            df = df.withColumn(
                column_name,
                when(trim(col(column_name)) == "", None)
                .otherwise(col(column_name))
            )

        else:
            raise ValueError(f"Unsupported operation: {operation}")

    return df


In [0]:
def write_delta_liquid(df, table_name, cluster_cols):
    (
        df.write
          .format("delta")
          .mode("overwrite")
          .clusterBy(*cluster_cols)
          .saveAsTable(table_name)
    )